In [3]:
import gzip
import pickle
from tensorflow import keras
import numpy as np
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, BatchNormalization, \
                                    Activation, Dropout, MaxPooling2D

def evaluate(path, model):
    X, y = pickle.load(gzip.open(path, 'rb'))
    y[y != 0] -= 2
    X = X / 255.
    acc = np.mean(model(X).numpy().argmax(axis=1) == y)
    return acc


In [4]:
path = 'flatland_train.data'
X,y = pickle.load(gzip.open(path,'rb')) 

# data pre-proc
y[y != 0] -= 2
X = X.reshape (X.shape[0],50,50,1) / 255.

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)

DNN modelis

In [6]:
# model = keras.models.Sequential()
# model.add(keras.layers.Flatten(input_shape=[50, 50]))
# model.add(keras.layers.Dense(64, activation="relu"))
# model.add(keras.layers.Dense(32, activation="relu"))
# model.add(keras.layers.Dense(10, activation="softmax"))
# model.compile(loss="sparse_categorical_crossentropy",
#               optimizer="Adam",
#               metrics=["accuracy"])
# model.summary()

CNN modelis

In [13]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[50, 50, 1]))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(Flatten())
model.add(Dropout(0.5))

model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_4 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 22, 22, 64)        18496     
                                                                 
 batch_normalization_5 (Batc  (None, 22, 22, 64)       256       
 hNormalization)                                                 
                                                      

In [14]:
loss = model.fit(X_train, y_train, epochs=7,validation_data=[X_test,y_test])


Epoch 1/7
219/219 [==============================] - 5s 17ms/step - loss: 1.0494 - accuracy: 0.6061 - val_loss: 4.9932 - val_accuracy: 0.1520
Epoch 2/7
219/219 [==============================] - 3s 15ms/step - loss: 0.3671 - accuracy: 0.8881 - val_loss: 7.2125 - val_accuracy: 0.1520
Epoch 3/7
219/219 [==============================] - 3s 15ms/step - loss: 0.2371 - accuracy: 0.9413 - val_loss: 1.4278 - val_accuracy: 0.5143
Epoch 4/7
219/219 [==============================] - 3s 15ms/step - loss: 0.1873 - accuracy: 0.9589 - val_loss: 0.2193 - val_accuracy: 0.9560
Epoch 5/7
219/219 [==============================] - 3s 15ms/step - loss: 0.1509 - accuracy: 0.9697 - val_loss: 0.1938 - val_accuracy: 0.9677
Epoch 6/7
219/219 [==============================] - 3s 15ms/step - loss: 0.1302 - accuracy: 0.9703 - val_loss: 0.2129 - val_accuracy: 0.9597
Epoch 7/7
219/219 [==============================] - 3s 15ms/step - loss: 0.1221 - accuracy: 0.9741 - val_loss: 0.1723 - val_accuracy: 0.9810


In [15]:
model.save('model_cnn25.h5')


In [16]:
model = keras.models.load_model('model_cnn25.h5')
evaluate('flatland_train.data', model)

0.9836

In [17]:
pred = model.predict(X_test).argmax(axis=1)
print('Accuracy on test set - {0:.02%}'.format((pred == y_test).mean()))

Accuracy on test set - 98.10%
